Download spacy library and polish language model

In [ ]:
!pip install --upgrade spacy
!python -m spacy download pl_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2022-09-15 14:24:06.853542: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 20.1 MB 1.3 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pl_core_news_sm')


Import necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import spacy
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import model_selection, naive_bayes, svm
from sklearn.compose import ColumnTransformer
from sklearn import metrics
from sklearn.preprocessing import FunctionTransformer
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.decomposition import NMF
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

Load polish language model

In [ ]:

nlp = spacy.load("pl_core_news_sm")
STOP_WORDS = spacy.lang.pl.STOP_WORDS

Load dataset and make basic checks on counts

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/Licencjat/DF_rich.xlsx")

In [ ]:
df = df.sort_values("type", ascending= True)

In [ ]:
df

,author,raw,ADU,type,target,ESE,Polarity,ASD-person,NO-person
0,Robert Biedroń,nie zajmują się zdobywaniem funduszy dla Polski,PiS i PO nie zajmują się zdobywaniem funduszy...,attack,Małgorzata Kidawa-Błońska,1.0,0.0,PiS i PO nie zajmują się zdobywaniem funduszy...,PiS i PO nie zajmują się zdobywaniem funduszy...
236,Władysław Kosiniak-Kamysz,podczas pana kadencji wiek emerytalny rolników...,podczas kadencji Andrzeja Dudy wiek emerytalny...,attack,Andrzej Duda,1.0,1.0,podczas kadencji Andrzeja Dudy wiek emerytalny...,podczas kadencji Andrzeja Dudy wiek emerytalny...
237,Władysław Kosiniak-Kamysz,Byłby pan niezłym archiwistą,Andrzej Duda byłby niezłym archiwistą,attack,Andrzej Duda,1.0,0.0,ASD byłby niezłym archiwistą,byłby niezłym archiwistą
239,Paweł Tanajno,"Śmieszy mnie wypowiedź Pana Andrzeja, który mó...","Paweł Tanajno śmieszy wypowiedź Pana Andrzeja,...",attack,Andrzej Duda,1.0,1.0,"ASD śmieszy wypowiedź Pana Andrzeja, który mów...","śmieszy wypowiedź Pana Andrzeja, który mówi, ..."
241,Paweł Tanajno,"przez ostatnie miesiące, razem z Mateuszem Mor...","przez ostatnie miesiące, razem z Mateuszem Mor...",attack,Mateusz Morawiecki,1.0,-1.0,"przez ostatnie miesiące, razem z Mateuszem Mor...","przez ostatnie miesiące, razem z Mateuszem Mor..."
...,...,...,...,...,...,...,...,...,...
1613,Marek Jakubiak,"tak, proszę państwa, wszyscy wiemy, że nie ma ...","wszyscy wiemy, że nie ma nic ważniejszego dla ...",NaN,NaN,0.0,2.0,"wszyscy wiemy, że nie ma nic ważniejszego dla ...","wszyscy wiemy, że nie ma nic ważniejszego dla ..."
1614,Mirosław Piotrowski,podnosił on sprawę szczepień i pytał: „A na kt...,"profesor, lekarz odpowiedzialn za koronawirusa...",NaN,NaN,0.0,0.0,"profesor, lekarz odpowiedzialn za koronawirusa...","profesor, lekarz odpowiedzialn za koronawirusa..."
1615,Mirosław Piotrowski,"wprowadzić w ogóle szczepionkę, to potrzebne s...","wprowadzić w ogóle szczepionkę, to potrzebne s...",NaN,NaN,0.0,0.0,"wprowadzić w ogóle szczepionkę, to potrzebne s...","wprowadzić w ogóle szczepionkę, to potrzebne s..."
1616,Mirosław Piotrowski,są tu też wśród nas lekarze i chyba potwierdzą,są tu też wśród nas lekarze i chyba potwierdzą,NaN,NaN,0.0,0.0,są tu też wśród nas lekarze i chyba potwierdzą,są tu też wśród nas lekarze i chyba potwierdzą


In [ ]:
df["type"].value_counts()

support    304
attack     200
Name: type, dtype: int64

In [ ]:
#I specifically choose data to make 50/50 ESE+/ESE- to balance the dataset
X = df[:400]['ASD-person'].copy().reset_index(drop=True).dropna()
Z = df[:400]['raw'].copy().reset_index(drop=True).dropna()
y = df[:400]['type'].copy().reset_index(drop=True).dropna()
# 

In [ ]:
y.describe()

count        400
unique         2
top       attack
freq         200
Name: type, dtype: object

Create lemmatization and part of speech functions

In [ ]:
def spacy_lemmatizer(X):
  X = X.apply(lambda row: " ".join([w.lemma_ for w in nlp(row) if not w.is_punct]))
  return X


def spacy_pos(X):
  X = X.apply(lambda row: " ".join([w.pos_ for w in nlp(row)]))
  return X



Create train and test datasets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.333, random_state = 42)

In [ ]:
X_train

119    w ogóle warto byłoby zakończyć współpracę z Un...
194                może nam ASD powie, kiedy będą wybory
229    ASD zapowiedział wczoraj, że w ramach rozdział...
265    ASD rozmawiał ostatnio z panem Michałem z podl...
218                     faktycznie pytanie jest o ustrój
                             ...                        
71     traktowani jesteśmy jak właśnie taki piesek na...
106    polscy przedsiębiorcy teraz są poddani eksterm...
270             potrzebne jest przede wszystkim zaufanie
348    jako  ASD chciałby się zająć, już w pierwszym ...
102                to nie było wynegocjonowane w dialogu
Name: ASD-person, Length: 266, dtype: object


# Lemma analysis and machine learning classification

Create dummy classifier for baseline

In [ ]:
dummy_clf = DummyClassifier(strategy="uniform")

In [ ]:
dummy_clf.fit(X_train,y_train)
pred=dummy_clf.predict(X_test)
print("Accuracy = "+str(metrics.accuracy_score(y_test, pred)))
print("Precision = "+str(metrics.precision_score(y_test, pred,pos_label="attack")))
print("Recall = "+str(metrics.recall_score(y_test, pred,pos_label="attack")))
# f1 score
print("F1 score = "+ str(f1_score(y_test, pred,pos_label="attack")))

Accuracy = 0.5522388059701493
Precision = 0.5641025641025641
Recall = 0.6285714285714286
F1 score = 0.5945945945945945


Create SVC model. Train it on lemmatized data and test the score.

In [ ]:
#pierwszy algorytm SVC; liczba komponentów 500?
text_clf_svc = Pipeline([("custom_function spacy", FunctionTransformer(spacy_lemmatizer)), ('tfidf', TfidfVectorizer()),('PCA-TruncatedSVD', TruncatedSVD(n_components=500)), ('clf', svm.SVC(probability=True))])
text_clf_svc.fit(X_train,y_train)
predictions = text_clf_svc.predict(X_test)
predictions_train_TOK_SVC = text_clf_svc.predict_proba(X_train)
predictions_test_TOK_SVC = text_clf_svc.predict_proba(X_test)
print(metrics.confusion_matrix(y_test, predictions))
# Print the overall accuracy
print("Accuracy = "+str(metrics.accuracy_score(y_test, predictions)))
print("Precision = "+str(metrics.precision_score(y_test, predictions, pos_label = "attack")))
print("Recall = "+str(metrics.recall_score(y_test, predictions, pos_label = "attack")))
# f1 score
print("F1 score = "+ str(f1_score(y_test, predictions, pos_label = "attack")))

[[48 22]
 [22 42]]
Accuracy = 0.6716417910447762
Precision = 0.6857142857142857
Recall = 0.6857142857142857
F1 score = 0.6857142857142857


Create Multinomial Naive Bayes model. Train it on lemmatized data and test the score.

In [ ]:
#Multinomial NaiveBayes
text_clf_nb = Pipeline([("custom_function spacy", FunctionTransformer(spacy_lemmatizer)),('tfidf', TfidfVectorizer()), ('clf', MultinomialNB(alpha=0.01))])
text_clf_nb.fit(X_train,y_train)
predictions_train_TOK_NB = text_clf_nb.predict_proba(X_train)
predictions_test_TOK_NB = text_clf_nb.predict_proba(X_test)
predictions = text_clf_nb.predict(X_test)
print(metrics.confusion_matrix(y_test, predictions))
# Print the overall accuracy
print("Accuracy = "+str(metrics.accuracy_score(y_test, predictions)))
print("Precision = "+str(metrics.precision_score(y_test, predictions, pos_label = "attack")))
print("Recall = "+str(metrics.recall_score(y_test, predictions, pos_label = "attack")))
# f1 score
print("F1 score = "+ str(f1_score(y_test, predictions, pos_label = "attack")))

[[48 22]
 [28 36]]
Accuracy = 0.6268656716417911
Precision = 0.631578947368421
Recall = 0.6857142857142857
F1 score = 0.6575342465753424


Create Maximum Entrophy model. Train it on lemmatized data and test the score.

In [ ]:
#trzeci Maximum Entrophy
text_clf_RF = Pipeline([('tfidf', TfidfVectorizer()),('clf', LogisticRegression(class_weight='balanced',penalty = 'none'))])
text_clf_RF.fit(X_train,y_train)
predictions = text_clf_RF.predict(X_test)
predictions_train_TOK_EN = text_clf_RF.predict_proba(X_train)
predictions_test_TOK_EN = text_clf_RF.predict_proba(X_test)
#print(metrics.confusion_matrix(y_test, predictions))
# Print the overall accuracy
print("Accuracy = "+str(metrics.accuracy_score(y_test, predictions)))
print("Precision = "+str(metrics.precision_score(y_test, predictions, pos_label = "attack")))
print("Recall = "+str(metrics.recall_score(y_test, predictions, pos_label = "attack")))
# f1 score
print("F1 score = "+ str(f1_score(y_test, predictions, pos_label = "attack")))

Accuracy = 0.7089552238805971
Precision = 0.7066666666666667
Recall = 0.7571428571428571
F1 score = 0.7310344827586207



# POS analysis and machine learning classification

Create SVC model. Train it on POS data and test the score.

In [ ]:
#SVC
text_clf_svc = Pipeline([("custom_function spacy", FunctionTransformer(spacy_pos)),('tfidf', TfidfVectorizer()), ('clf', svm.SVC(probability=True))])
text_clf_svc.fit(X_train,y_train)
predictions = text_clf_svc.predict(X_test)
predictions_train_POS_SVC = text_clf_svc.predict_proba(X_train)
predictions_test_POS_SVC = text_clf_svc.predict_proba(X_test)
print(metrics.confusion_matrix(y_test, predictions))
# Print the overall accuracy
print("Accuracy = "+str(metrics.accuracy_score(y_test, predictions)))
print("Precision = "+str(metrics.precision_score(y_test, predictions, pos_label = "attack")))
print("Recall = "+str(metrics.recall_score(y_test, predictions, pos_label = "attack")))
# f1 score
print("F1 score = "+ str(f1_score(y_test, predictions, pos_label = "attack")))

[[28 42]
 [13 51]]
Accuracy = 0.5895522388059702
Precision = 0.6829268292682927
Recall = 0.4
F1 score = 0.5045045045045045


Create Multinomial Naive Bayes model. Train it on POS data and test the score.

In [ ]:
#MultinomialNB 
text_clf_nb = Pipeline([("custom_function spacy", FunctionTransformer(spacy_pos)),('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])
text_clf_nb.fit(X_train,y_train)
predictions = text_clf_nb.predict(X_test)
predictions_train_POS_NB = text_clf_nb.predict_proba(X_train)
predictions_test_POS_NB = text_clf_nb.predict_proba(X_test)
print(metrics.confusion_matrix(y_test, predictions))
# Print the overall accuracy
print("Accuracy = "+str(metrics.accuracy_score(y_test, predictions)))
print("Precision = "+str(metrics.precision_score(y_test, predictions, pos_label = "attack")))
print("Recall = "+str(metrics.recall_score(y_test, predictions, pos_label = "attack")))
# f1 score
print("F1 score = "+ str(f1_score(y_test, predictions, pos_label = "attack")))

[[28 42]
 [17 47]]
Accuracy = 0.5597014925373134
Precision = 0.6222222222222222
Recall = 0.4
F1 score = 0.4869565217391305


Create Maximum Entrophy model. Train it on POS data and test the score.

In [ ]:
#Maximum Entrophy
text_clf_svc = Pipeline([("custom_function spacy", FunctionTransformer(spacy_pos)),('tfidf', TfidfVectorizer()), ('clf', LogisticRegression( solver = 'liblinear'))])
text_clf_svc.fit(X_train,y_train)
predictions = text_clf_svc.predict(X_test)
predictions_train_POS_EN = text_clf_svc.predict_proba(X_train)
predictions_test_POS_EN = text_clf_svc.predict_proba(X_test)
print(metrics.confusion_matrix(y_test, predictions))
# Print the overall accuracy
print("Accuracy = "+str(metrics.accuracy_score(y_test, predictions)))
print("Precision = "+str(metrics.precision_score(y_test, predictions, pos_label = "attack")))
print("Recall = "+str(metrics.recall_score(y_test, predictions, pos_label = "attack")))
# f1 score
print("F1 score = "+ str(f1_score(y_test, predictions, pos_label = "attack")))

[[34 36]
 [17 47]]
Accuracy = 0.6044776119402985
Precision = 0.6666666666666666
Recall = 0.4857142857142857
F1 score = 0.5619834710743802


#Meta models using "stacking" - that means joining results from previous lemma and POS classification and using another model to predict class based on the results of previous models.

Create meta model train and test set

In [ ]:
new_train=pd.merge(pd.DataFrame(predictions_train_POS_EN[:,0],columns=['POS']),pd.DataFrame(predictions_train_TOK_EN[:,0],columns=['ADU']),left_index=True, right_index=True)
new_test = pd.merge(pd.DataFrame(predictions_test_POS_EN[:,0],columns=['POS']),pd.DataFrame(predictions_test_TOK_EN[:,0],columns=['ADU']),left_index=True, right_index=True)

In [ ]:
new_train

,POS,ADU
0,0.398763,9.999902e-01
1,0.621809,9.999885e-01
2,0.388300,1.858415e-07
3,0.357963,2.639045e-06
4,0.331062,9.253205e-07
...,...,...
261,0.543435,1.000000e+00
262,0.371840,9.999993e-01
263,0.406486,1.174838e-12
264,0.600119,3.944924e-10


Create SVC model. Train it on meta data and test the score.

In [ ]:
#SVC
text_clf_svc_new = svm.SVC(random_state=432, class_weight='balanced')
text_clf_svc_new.fit(new_train,y_train)
predictions = text_clf_svc_new.predict(new_test)
print(metrics.confusion_matrix(y_test, predictions))
# Print the overall accuracy
print("Accuracy = "+str(metrics.accuracy_score(y_test, predictions)))
print("Precision = "+str(metrics.precision_score(y_test, predictions, pos_label = "attack")))
print("Recall = "+str(metrics.recall_score(y_test, predictions, pos_label = "attack")))
# f1 score
print("F1 score = "+ str(f1_score(y_test, predictions, pos_label = "attack")))

[[53 17]
 [22 42]]
Accuracy = 0.7089552238805971
Precision = 0.7066666666666667
Recall = 0.7571428571428571
F1 score = 0.7310344827586207


Create Multinomial Naive Bayes model. Train it on POS data and test the score.

In [ ]:
#NB
text_clf_svc_new = MultinomialNB()
text_clf_svc_new.fit(new_train,y_train)
predictions = text_clf_svc_new.predict(new_test)
print(metrics.confusion_matrix(y_test, predictions))
# Print the overall accuracy
print("Accuracy = "+str(metrics.accuracy_score(y_test, predictions)))
print("Precision = "+str(metrics.precision_score(y_test, predictions, pos_label = "attack")))
print("Recall = "+str(metrics.recall_score(y_test, predictions, pos_label = "attack")))
# f1 score
print("F1 score = "+ str(f1_score(y_test, predictions, pos_label = "attack")))

[[57 13]
 [25 39]]
Accuracy = 0.7164179104477612
Precision = 0.6951219512195121
Recall = 0.8142857142857143
F1 score = 0.75


Create Maximum Entrophy model. Train it on meta data and test the score.

In [ ]:
#Maximum Entrophy
text_clf_svc_new = LogisticRegression(class_weight='balanced', penalty = 'none', random_state=4132)
text_clf_svc_new.fit(new_train,y_train)
predictions = text_clf_svc_new.predict(new_test)
print(metrics.confusion_matrix(y_test, predictions))
# Print the overall accuracy
print("Accuracy = "+str(metrics.accuracy_score(y_test, predictions)))
print("Precision = "+str(metrics.precision_score(y_test, predictions, pos_label = "attack")))
print("Recall = "+str(metrics.recall_score(y_test, predictions, pos_label = "attack")))
# f1 score
print("F1 score = "+ str(f1_score(y_test, predictions, pos_label = "attack")))

[[53 17]
 [22 42]]
Accuracy = 0.7089552238805971
Precision = 0.7066666666666667
Recall = 0.7571428571428571
F1 score = 0.7310344827586207
